In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
df_1 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/clean/df_merged.csv')

In [ ]:
df_2 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_1.txt')
df_3 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_2.txt')

In [ ]:
merged_df = pd.concat([df_2, df_3], axis=0)

In [ ]:
df_merged = df_1.merge(merged_df, on='client_id', how='inner')
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])
df_merged.dtypes

In [ ]:
df_merged.to_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/clean/combined_cleaned_data.csv', index=False)

In [ ]:
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

In [ ]:
# Sort control group
control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

# Sort test group
test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

In [ ]:
test_group_sorted.head()

In [ ]:
# Function to get latest starts
def filter_latest_starts(group_df):
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]
    # Merge back to get the full process after the last start
    return df_merged.merge(latest_starts[['client_id', 'date_time']], on=['client_id', 'date_time'], how='inner')

# Apply to both groups
filtered_control = filter_latest_starts(control_group)
filtered_test = filter_latest_starts(test_group)


In [ ]:
client_total_entries = df_merged[df_merged["client_id"] == 2304905]
client_total_entries

In [ ]:
client_last_start = filtered_control[filtered_control['client_id'] == 2304905 ]
client_last_start

In [ ]:
# Step 1: Filter to get the latest start for each client
starts_only = df_merged[df_merged['process_step'] == 'start']
latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

# Step 2: Filter to get the last confirmation for each client
confirmation_only = df_merged[df_merged['process_step'] == 'confirm']
latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

# Step 2: Merge to have both latest start and confirm per client
# Confirming datetime columns are of Timestamp type in merged DataFrame
latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

# Step 3: Calculate process duration for those who completed the process
latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

# Step 4: Calculate the average duration
average_duration = latest_start_confirms['process_duration'].mean()
latest_start_confirms

In [ ]:
latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))


In [ ]:
latest_start_confirms

In [ ]:
latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']


In [ ]:
# Function to calculate completion rates
def calculate_completion_rate(group):
    started = group.groupby('process_step')['client_id'].nunique()
    completed = group.groupby('process_step').apply(lambda x: x[x['process_step'] != 'start']['client_id'].nunique())
    completion_rate = (completed / started).fillna(0) * 100  # Convert to percentage
    return completion_rate.reset_index(name='completion_rate')

# Calculate for control and test groups
control_completion_rate = calculate_completion_rate(control_group_sorted)
test_completion_rate = calculate_completion_rate(test_group_sorted)

In [ ]:
# Merge control and test completion rates
merged_completion_rates = control_completion_rate.merge(
    test_completion_rate, on='process_step', suffixes=('_control', '_test')
)

# Perform chi-square test for each step
for _, row in merged_completion_rates.iterrows():
    observed = [row['completion_rate_control'], row['completion_rate_test']]
    chi2, p = chi2_contingency([observed])
    print(f"Step: {row['process_step']}, p-value: {p}")